## 1. Problem Background

Try and predict how much money to budget for sales commissions. 
We have a defined formula for calculating commissions and we likely have some experience with prior year's commissions payments.

![ChessUrl](https://pbpython.com/images/Excel_Table_1.png "chess")

Commission Amount = Actual Sales * Commission Rate

Where commission rate is the percentage or fixed payment associated with a certain amount of sale. Usually a service charge assessed by a broker or investment advisor.

Commission rate is based on the percent to plan table:
E.g. how much of their target they met.
![ChessUrl](https://pbpython.com/images/Excel_Table_4.png "chess")

For Monte Carlo you can iterate through each commission rate for each sales person to determine a range of outcomes.

There are two components to Monte Carlo:
- The equation to evaluate
- The random variables for input

![ChessUrl](https://pbpython.com/images/monte_carlo_image_hist_pct.png "chess")

Normal distribution with a mean of 100% and standard deviation of 10%.
We can model our input variable didstribution so that it is similar to our real world experience.


## Building a Normal Distribution of the % to Target Distribution
Using pandas to construct a model that replicates the excel spreadsheet calculation.

In [33]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set_style('whitegrid')

In [34]:
#Use random numper generation for numpy.
#Numpy has several random number generator that can create random samples based on a 
#predefined distribution

#mean of 100%
avg = 1

#std_dev of 10%
std_dev =.1

#500 sales reps
num_reps = 500


In [35]:
#Use numpy to generate a list of percentages that will replicate the historical
#normal distribution

pct_to_target = np.random.normal(avg, std_dev, num_reps).round(2)
#rounded to 2 decimal places
pct_to_target[:10]

array([0.87, 1.05, 0.91, 1.08, 0.97, 0.94, 1.06, 1.12, 0.91, 0.85])

## Building a Scaled Uniform Distribution for the Historic Sales Target Distribution

The historic sales targets show that sales targets are in 1 of 6 buckets and the frequency decreases as the amount increases. E.g. individuals could be bucketed into certain groups and given targets based on tenure, terroritoy or sales pipelines.

![](https://pbpython.com/images/monte_carlo_sales_target.png "sales_targets")

In [36]:
#Use nump.random.choice to recreate the historic sales target distribution

sales_target_values = [75000, 100000, 200000, 300000, 400000, 500000]
sales_target_prob = [.3, .3, .2, .1, .05, .05]
#The choice() method returns a randomly selected element from the specified sequence.
sales_target = np.random.choice(sales_target_values, num_reps, p=sales_target_prob)
sales_target[:10]

array([100000, 200000, 100000,  75000,  75000,  75000, 500000, 200000,
       100000, 100000])

## Build a Dataframe Containing Both Distribution
Both distributions (sales_target and pct_to_target contain 500 sales reps with randomly distributed values

In [37]:
df = pd.DataFrame(index=range(num_reps), data={'Pct_To_Target': pct_to_target,
                                              'Sales_Target': sales_target})
df['Sales'] = df['Pct_To_Target'] * df['Sales_Target']
df.head()

,Pct_To_Target,Sales_Target,Sales
0,0.87,100000,87000.0
1,1.05,200000,210000.0
2,0.91,100000,91000.0
3,1.08,75000,81000.0
4,0.97,75000,72750.0


## Map the Pct_To_Target to the Commission Rate

Looking to copy the table below:

![](https://pbpython.com/images/Excel_Table_4.png 'Commision Rate Table')

In [38]:
def calc_commission_rate(x):
    """Returns the commisision rate based on the above table"""
    if x <= .9:
        return .02
    if x <= .99:
        return .03
    else:
        return .04

In [39]:
#Apply the above function to pct_target and then calculate the commission amount
df['Commission_Rate'] = df['Pct_To_Target'].apply(calc_commission_rate)
df['Commission_Amount'] = df['Commission_Rate'] * df['Sales']
df.head()

,Pct_To_Target,Sales_Target,Sales,Commission_Rate,Commission_Amount
0,0.87,100000,87000.0,0.02,1740.0
1,1.05,200000,210000.0,0.04,8400.0
2,0.91,100000,91000.0,0.03,2730.0
3,1.08,75000,81000.0,0.04,3240.0
4,0.97,75000,72750.0,0.03,2182.5


## Loop the Sales Table to Simulate the Outcome

Basically, now repeat what was written down previously in calculating the normal and scaled uniform distributions to calculate the sales_targets and the percent to target

In [40]:
#this list will hold all of the outputs
all_stats = []

#1000 simulations
num_simulations = 10000

for i in range (num_simulations):
    
    #Generate distribution
    sales_target = np.random.choice(sales_target_values, num_reps, p=sales_target_prob)
    pct_to_target = np.random.normal(avg, std_dev, num_reps).round(2)
    
    #Build the dataframe
    df = pd.DataFrame(index=range(num_reps), data={'Pct_To_Target':pct_to_target,
                                                  'Sales_Target':sales_target})
    
    #Calculate sales, Commission rate and Commission amount
    df['Sales'] = df['Pct_To_Target'] * df['Sales_Target']
    df['Commission_Rate'] = df['Sales'].apply(calc_commission_rate)
    df['Commission_Amount'] = df['Commission_Rate'] * df['Sales']
    
    #Track the sales, commission amounts and sales targets
    all_stats.append([df['Sales'].sum().round(0),
                    df['Commission_Amount'].sum().round(0),
                    df['Sales_Target'].sum().round(0)])

In [41]:
all_stats[:5]

[[81077750.0, 3243110.0, 81125000],
 [82114500.0, 3284580.0, 80975000],
 [81690500.0, 3267620.0, 82100000],
 [84240500.0, 3369620.0, 83700000],
 [82499750.0, 3299990.0, 82450000]]

In [42]:
#Load all the stats into a dataframe and look at the stats
#From records will convert a structed array into a dataframe
results_df = pd.DataFrame.from_records(all_stats, columns=['Sales',
                                                          'Commission_Amount',
                                                          'Sales_Target'])

results_df.describe().style.format('{:,}')

,Sales,Commission_Amount,Sales_Target
count,"10,000.0","10,000.0","10,000.0"
mean,"83,715,013.175","3,348,600.527","83,716,942.5"
std,"2,685,451.5327201067","107,418.06130880286","2,643,384.9062460223"
min,"73,138,000.0","2,925,520.0","73,175,000.0"
25%,"81,868,250.0","3,274,730.0","81,925,000.0"
50%,"83,692,750.0","3,347,710.0","83,700,000.0"
75%,"85,531,812.5","3,421,272.5","85,481,250.0"
max,"94,012,750.0","3,760,510.0","94,600,000.0"
